In [ ]:
# 简单的训练一个分词器
import sentencepiece as spm

input = '/Users/luxun/workspace/ai/mine/llm_pretrain/.temp/the-verdict.txt'
spm.SentencePieceTrainer.train(input=input, model_prefix='m', vocab_size=1000, user_defined_symbols=['foo', 'bar'],
                               model_type='bpe')

In [ ]:
#使用训练好的分词器
sp = spm.SentencePieceProcessor()
sp.load('m.model')
id = sp.encode('hello world')
print(id)
s = sp.decode(id)
s

In [ ]:
# 加载一个tiktoken分词器（）
import tiktoken

enc = tiktoken.get_encoding("o200k_base")
assert enc.decode(enc.encode("hello world")) == "hello world"
# To get the tokeniser corresponding to a specific model in the OpenAI API:
enc = tiktoken.encoding_for_model("gpt-4o")
enc.encode("aa d")

In [ ]:
# 加载一个transformers分词器
from transformers import PreTrainedTokenizerFast

# 加载预训练分词器，这里以 BERT 的分词器为例
tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="/Users/luxun/workspace/ai/mine/llm_pretrain/notebook/tokenizer.json")

# 输入文本
text = "这是一个中文句子."

# 1. 使用 tokenizer.tokenize
tokens = tokenizer.tokenize(text)
print("Tokenized result:", tokens)
# 输出: ['this', 'is', 'a', 'test', 'sentence', '.']

# 2. 使用 tokenizer.encode
token_ids = tokenizer.encode(text, add_special_tokens=True)
print("Encoded result (token IDs):", token_ids)
# 输出: [101, 2023, 2003, 1037, 3231, 6251, 1012, 102] （以 BERT 为例）

# 3. 解码 token IDs
decoded_text = tokenizer.decode(token_ids)
print("Decoded text:", decoded_text)
# 输出: "this is a test sentence."


In [ ]:
import chardet
from charset_normalizer import from_bytes


def detect_file_encoding(file_path):
    """
    检测文件编码。
    :param file_path: 文件路径
    :return: 编码类型
    """
    try:
        with open(file_path, "rb") as file:
            raw_data = file.read()

            # 方法 1: 使用 chardet 检测
            chardet_result = chardet.detect(raw_data)
            chardet_encoding = chardet_result['encoding']

            # 方法 2: 使用 charset-normalizer 检测
            charset_result = from_bytes(raw_data).best()
            charset_encoding = charset_result.encoding if charset_result else None

            # 返回检测结果
            return {
                "chardet_encoding": chardet_encoding,
                "charset_normalizer_encoding": charset_encoding
            }
    except Exception as e:
        return {"error": str(e)}


# 示例使用
file_path = "/Users/luxun/workspace/ai/mine/llm_pretrain/notebook/tokenizer.json"
result = detect_file_encoding(file_path)
print("Encoding detection result:", result)

In [3]:
# 使用tokenizers BPE训练一个分词器
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.normalizers import NFKC
from tokenizers.processors import TemplateProcessing

# 初始化 BPE 分词器
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

# 设置正则化器（可选）
tokenizer.normalizer = NFKC()

# 设置预分词器
tokenizer.pre_tokenizer = Whitespace()

# 定义 BPE 训练器
trainer = BpeTrainer(
    vocab_size=30000,  # 词汇表大小
    min_frequency=2,  # 最低频率
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],  # 特殊标记
)

# 加载训练数据
files = ["/Users/luxun/workspace/ai/mine/llm_pretrain/.temp/the-verdict.txt"]

# 开始训练
tokenizer.train(files, trainer)

# 设置 post-processing（例如模板处理）
tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", tokenizer.token_to_id("[CLS]")),
        ("[SEP]", tokenizer.token_to_id("[SEP]")),
    ],
)

# 保存分词器
tokenizer.save("tokenizer-bpe.json")

print("BPE 分词器训练完成并保存为 tokenizer-bpe.json")





BPE 分词器训练完成并保存为 tokenizer-bpe.json


In [5]:
from tokenizers import Tokenizer

# 加载分词器
tokenizer = Tokenizer.from_file("tokenizer-bpe.json")

# 对文本进行分词
output = tokenizer.encode("hello")
print("Tokens:", output.tokens)
print("IDs:", output.ids)


Tokens: ['[CLS]', 'he', 'll', 'o', '[SEP]']
IDs: [2, 65, 95, 53, 3]
